In [ ]:
import numpy as np
import pandas as pd
import torch
from options import Options
from attention_tcn import Attention_TCN_Model

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

import time
from utils import generate_data, pinball_loss, get_seperate_results, plot

In [ ]:
all_data = pd.read_csv('Wavelet_data.csv')
opt = Options()

electricity = torch.tensor(all_data['KW']).type(torch.float32).unsqueeze(0)
# cold = torch.tensor
cold = torch.tensor(all_data['CHWTON']).type(torch.float32).unsqueeze(0)
heat = torch.tensor(all_data['HTmmBTU']).type(torch.float32).unsqueeze(0)
DPT = torch.tensor(all_data['HourlyDewPointTemperature']).type(torch.float32).unsqueeze(0)
DBT = torch.tensor(all_data['HourlyDryBulbTemperature']).type(torch.float32).unsqueeze(0)
WBT = torch.tensor(all_data['HourlyWetBulbTemperature']).type(torch.float32).unsqueeze(0)
RH = torch.tensor(all_data['HourlyRelativeHumidity']).type(torch.float32).unsqueeze(0)
SP = torch.tensor(all_data['HourlyStationPressure']).type(torch.float32).unsqueeze(0)

three_loads = torch.cat((electricity, cold, heat), 0)
weather_data = torch.cat((DPT,DBT,WBT,RH,SP),0)
three_loads_hour, three_loads_y = generate_data(three_loads, opt.lagged_hour)
weather_hour, weather_y = generate_data(weather_data, opt.lagged_hour)

three_loads_hour_train = three_loads_hour[:int(opt.train_rate*three_loads_hour.size(0)), :, :]
three_loads_hour_test = three_loads_hour[int(opt.train_rate*three_loads_hour.size(0)):, :, :]

weather_hour_train = weather_hour[:int(opt.train_rate*weather_hour.size(0)), :, :]
weather_hour_test = weather_hour[int(opt.train_rate*weather_hour.size(0)):, :, :]

three_loads_y_train = three_loads_y[:int(opt.train_rate*three_loads_hour.size(0)), :]
three_loads_y_test =  three_loads_y[int(opt.train_rate*three_loads_hour.size(0)):, :]

train_dataset = TensorDataset(three_loads_hour_train.to(opt.device), weather_hour_train.to(opt.device), three_loads_y_train.to(opt.device))
train_dataloader = DataLoader(train_dataset, opt.batch, shuffle=True)

In [ ]:
att_TCN_Model = Attention_TCN_Model(opt.attention_input_size, opt.attention_embed_dim, opt.num_heads, opt.tcn_hidden_size, opt.num_loads, opt.num_blocks, opt.num_weather).to(opt.device)
optimizer = torch.optim.Adam(att_TCN_Model.parameters(), lr = opt.learning_rate)


start_time = time.time()
train_loss = []
for i in range(opt.n_epochs):
    i += 1
    optimizer.zero_grad()
    loss_list_train = []
    for batch_load, batch_weather, batch_labels in train_dataloader:
        batch_quantile_loss = []
        outputs = att_TCN_Model(batch_load, batch_weather)
        for quantile in opt.quantiles:
            batch_quantile_loss.append(pinball_loss(quantile, outputs[opt.quantiles.index(quantile)], batch_labels))
        batch_loss = sum(batch_quantile_loss)/len(batch_quantile_loss)
        loss_list_train.append(batch_loss)
        
    loss = sum(loss_list_train)/len(loss_list_train)
    train_loss.append(loss)
    
    loss.backward()
    optimizer.step()
    # torch.save(att_TCN_Model, 'train_models\model_epoch_{}'.format(i))
    
    if i%10 == 0:
        print('epoch: {}, train_loss: {}'.format(i, loss.item()))
    
end_time = time.time()
print('time cost', (end_time-start_time)/60,'min')
torch.save(att_TCN_Model, 'test')

In [ ]:
# att_TCN_Model = torch.load('test')
att_TCN_Model.eval()
three_loads_hour_test = three_loads_hour_test.to(opt.device)
weather_hour_test = weather_hour_test.to(opt.device)
att_TCN_Model_output = att_TCN_Model(three_loads_hour_test, weather_hour_test)

electricity_results, cold_results, heat_results = get_seperate_results(att_TCN_Model_output, three_loads_y_test, opt.quantiles)

electricity_results = electricity_results.sort(0, False)[0]
cold_results = cold_results.sort(0, False)[0]
heat_results = heat_results.sort(0, False)[0]

plot(three_loads_y_test, forecased_result=electricity_results, num_result=200, start=220, flag=0)